# LinkedIn Data Project

Today's labor force scours the internet looking for employers who will provide a good work envirnment. While website reviews are helpful to understand a company's vibe and Google images captures an owner's desired feng shui, accurate employee data reveals more about a work setting than anthing else. 

This project shows employee data for 100 companies in the Tech field, Automotive industy, and food/beverage manufacturing respectively. It will look for job descriptions, duration with a company, and region the company is located, amoung other things. It will take this raw data and deliver it in an understandable Tableau dashboard. In so doing it will allow people to navigate their careers through informed decision making. 

The companies we will look at will be a sampling of their individual markets and the employee info gathered will be a sampling of the individual company. This will give us a normal distribution of how each company behaves. 

This can all be accomplished in three main steps:

## Step One: Gather Company URLs

All three hundred companies we will look at were found at these websites: https://www.foodprocessing.com/top100/2020/, https://www.forbes.com/top-digital-companies/list/#tab:rank, and https://www.berylls.com/wp-content/uploads/2020/07/202007_BERYLLS_Study_Top_100_supplier-2019_EN.pdf. 

The data was taken and cleaned up on Google sheets to allow it to be used as lists in Python. Using these lists we used the Selemium package to gather the company URLs. 

The strategy used here is to get all the hrefs as we search each company and filter to get a unique list of urls with '/company/'

In [ ]:
import time
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

#log on to LinkedIn and access page
driver.get("https://www.linkedin.com/company/tesla-motors/people/")
driver.find_element_by_xpath("/html[1]/body[1]/div[1]/main[1]/div[2]/div[1]/form[1]/div[1]/input[1]").send_keys("********")
driver.find_element_by_xpath("/html[1]/body[1]/div[1]/main[1]/div[2]/div[1]/form[1]/div[2]/input[1]").send_keys("********")
driver.find_element_by_xpath("//button[contains(text(),'Sign in')]").click()

time.sleep(2)

auto_href_list = []
refined_auto = []
auto_company_list = []
final_auto = []
substring = '/company/'
substring2 = 'jobs'
for company in tech_list:
    driver.get('https://www.linkedin.com/company/tesla-motors/people/')
    #Clicking on the search bar
   
    search = driver.find_element_by_xpath("//*[@id='global-nav-typeahead']/input")
    search.click()
    time.sleep(1)
    search.send_keys(company)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="global-nav-typeahead"]/input').send_keys(Keys.ENTER)
    #driver.find_element_by_xpath("//div[@id='global-nav-typeahead']").send_keys('hi')
    time.sleep(1)
    hrefs = driver.find_elements_by_xpath("//a[@href]")
    for href in hrefs:
        auto_href_list.append(href.get_attribute("href")) 
    for url in auto_href_list:
        if substring in url and substring2 not in url:
            auto_company_list.append(url)
    for reference in auto_company_list:
        if reference not in refined_auto:
            refined_auto.append(reference)
    for elem in refined_auto:
        string_add = 'people/'
        elem = elem + string_add
        final_auto.append(elem)
    auto_href_list.clear()
    auto_company_list.clear()
    refined_auto.clear()   
auto_file = open('D:\SpringBoard Capstone Practice\\techcompanyurlList.txt','w')
for instance in final_auto:
    auto_file.write(instance+'\n')

auto_file.close()           

## Step Two: Retrieve Employee Profiles from Company URLs 

The strategy used here is to get the info by searching each company and getting the employee profiles in the hrefs with '/in/' in the url, while scrolling through the company employee page

In [ ]:
SCROLL_PAUSE_TIME = 3

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")
file = open("teslaprofiles.csv", 'w')
final_list = []
# https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python/27760083 (7/7/21) Cuong Tran
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    
    substring = "/in/"
   
    profiles_list = []
    links_list = []

    hrefs = driver.find_elements_by_xpath("//a[@href]")
    for href in hrefs:
        links_list.append(str(href.get_attribute("href")))
    for link in links_list:
        if substring in link:
            profiles_list.append(link)
    for profile in profiles_list:
        if profile not in final_list:
            final_list.append(profile)
            with open('file', 'w') as f:
                writer = csv.writer(f)
                writer.writerow(profile)
                print(profile)
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        file.close()
        break
    last_height = new_height
    hrefs.clear()
    profiles_list.clear()
    print(len(final_list))


## Step Three: Get Individual Employee data and send to CSV files

The strategy here is to search through each employee profile and extract publicly available info using xpath. 

In [ ]:
driver.get('https://www.linkedin.com/in/michael-adams-b7053914b/')

name = driver.find_element_by_xpath("//h1[contains(text(),'')]").text
employer = driver.find_element_by_xpath("//body/div[5]/div[3]/div[1]/div[1]/div[1]/div[1]/div[3]/div[1]/div[1]/main[1]/div[1]/section[1]/div[2]/div[2]/ul[1]/li[1]/a[1]/h2[1]/div[1]").text
job = driver.find_element_by_xpath("//body/div[5]/div[3]/div[1]/div[1]/div[1]/div[1]/div[3]/div[1]/div[1]/main[1]/div[1]/section[1]/div[2]/div[2]/div[1]/div[2]").text

exp = driver.find_element_by_xpath('/html/body/div[5]/div[3]/div/div/div/div/div[3]/div/div/main/div/div[5]/div[2]/span/div/section/div[1]/section/ul')

area = driver.find_element_by_css_selector("#ember331 > div.pv-entity__summary-info.pv-entity__summary-info--background-section.mb2 > div > h4:nth-child(2) > span.pv-entity__bullet-item-v2").text
